In [15]:
import psycopg2
import configparser
import pandas as pd
import numpy as np
portNum = 5903
#config = configparser.ConfigParser()
#config.read("../../config.ini")    
#db_params = dict(config['DB'])
from sqlalchemy import create_engine
db_params = {"dbname": "main", "user": "aurimas", "password": "aurimas6242", "host": "localhost", "port": portNum}
def execute_sql(SQL):
        with psycopg2.connect(**db_params) as conn:
            with conn.cursor() as cur:
                cur.execute(SQL)        


def get_alchemy_engine():
    conn_string = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**db_params)
    return create_engine(conn_string, echo=False)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoLars, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.metrics import mean_squared_error
df_accessibility = pd.read_sql_query("SELECT * FROM public.accessibility_stats", get_alchemy_engine())
df_h3demo = pd.read_sql_query("SELECT * FROM public.h3demographics", get_alchemy_engine())

In [17]:
df_accessibility

,id,cityid,categorytype,poi_category,timeofday,h3id,accessibility
0,3611383,4,Income,Vaccination centre,morning,8926645300fffff,3.642498
1,3611384,4,Income,Vaccination centre,morning,89266453013ffff,2.217098
2,3611385,4,Income,Vaccination centre,morning,89266453017ffff,0.321941
3,3611386,4,Income,Vaccination centre,morning,8926645301bffff,3.642498
4,3611387,4,Income,Vaccination centre,morning,89266453023ffff,2.879983
...,...,...,...,...,...,...,...
4615485,3611378,4,Income,Vaccination centre,morning,89266452eb3ffff,1.863840
4615486,3611379,4,Income,Vaccination centre,morning,89266452eb7ffff,1.863840
4615487,3611380,4,Income,Vaccination centre,morning,89266452ebbffff,0.766955
4615488,3611381,4,Income,Vaccination centre,morning,89266453003ffff,3.642498


In [19]:
df_h3demo

,cityid,categorytype,groupname,h3id,population,id
0,4,Age and Sex,Under 18 years female,8926645a963ffff,0.960630,1898645
1,4,Age and Sex,18 to 45 years female,8926645a963ffff,3.307087,1898646
2,4,Age and Sex,45 to 65 years female,8926645a963ffff,2.062992,1898647
3,4,Age and Sex,65 years and over female,8926645a963ffff,3.519685,1898648
4,4,Race,White,8926645a963ffff,15.574803,1898649
...,...,...,...,...,...,...
3419970,4,Race,American Indian and Alaska Native,8926645a963ffff,0.000000,1898640
3419971,4,Age and Sex,Under 18 years male,8926645a963ffff,1.133858,1898641
3419972,4,Age and Sex,18 to 45 years male,8926645a963ffff,1.834646,1898642
3419973,4,Age and Sex,45 to 65 years male,8926645a963ffff,1.543307,1898643


In [20]:
pops = {}
totalPops = {} #total population by h3id
groupnames = {}
for i in range(len(df_h3demo)):
    if(i%10000 == 0):
        print(i)
    row = df_h3demo.iloc[i]
    h3id = row['h3id']
    if(row['categorytype'] == "Age and Sex"):
        if(h3id not in totalPops):
            totalPops[h3id] = 0
        totalPops[h3id] += row['population']
    group = row['groupname']
    if(group not in groupnames):
        groupnames[group] = 1
    pops[(h3id, group)] = row['population']

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [21]:
print(groupnames)

{'Under 18 years female': 1, '18 to 45 years female': 1, '45 to 65 years female': 1, '65 years and over female': 1, 'White': 1, 'Black or African American': 1, 'Vehicle available': 1, 'No vehicle available': 1, '$200000 or more': 1, '$150000 to $200000': 1, '$75000 to $150000': 1, '$40000 to $75000': 1, '$10000 to $40000': 1, 'less than $10000': 1, 'Hispanic or Latino': 1, 'Not Hispanic or Latino': 1, 'Two or more races': 1, 'Some other race': 1, 'Native Hawaiian and Other Pacific Islander': 1, 'Asian': 1, 'American Indian and Alaska Native': 1, 'Under 18 years male': 1, '18 to 45 years male': 1, '45 to 65 years male': 1, '65 years and over male': 1}


In [22]:
for groupName in ['White', 'Under 18 years male', '65 years and over male', 'Vehicle available', 'less than $10000', '$200000 or more']:
    print(groupName)
    arr = []
    for i in range(len(df_accessibility)):
        h3id = df_accessibility.iloc[i]['h3id']
        arr.append(pops[(h3id, groupName)] / totalPops[h3id])
    df_accessibility['proportion_' + groupName] = arr

White


/var/folders/p4/_70b6r8579d_z47wctm8m73h0000gn/T/ipykernel_10209/1122640779.py:6: RuntimeWarning: invalid value encountered in double_scalars
  arr.append(pops[(h3id, groupName)] / totalPops[h3id])


Under 18 years male


/var/folders/p4/_70b6r8579d_z47wctm8m73h0000gn/T/ipykernel_10209/1122640779.py:6: RuntimeWarning: invalid value encountered in double_scalars
  arr.append(pops[(h3id, groupName)] / totalPops[h3id])


65 years and over male


...

/var/folders/p4/_70b6r8579d_z47wctm8m73h0000gn/T/ipykernel_10209/1122640779.py:6: RuntimeWarning: invalid value encountered in double_scalars
  arr.append(pops[(h3id, groupName)] / totalPops[h3id])


less than $10000


/var/folders/p4/_70b6r8579d_z47wctm8m73h0000gn/T/ipykernel_10209/1122640779.py:6: RuntimeWarning: invalid value encountered in double_scalars
  arr.append(pops[(h3id, groupName)] / totalPops[h3id])


$200000 or more


/var/folders/p4/_70b6r8579d_z47wctm8m73h0000gn/T/ipykernel_10209/1122640779.py:6: RuntimeWarning: invalid value encountered in double_scalars
  arr.append(pops[(h3id, groupName)] / totalPops[h3id])


In [23]:
df_accessibilit

,id,cityid,categorytype,poi_category,timeofday,h3id,accessibility,proportion_White,proportion_Under 18 years male,proportion_65 years and over male,proportion_Vehicle available,proportion_less than $10000,proportion_$200000 or more
0,3611383,4,Income,Vaccination centre,morning,8926645300fffff,3.642498,0.956320,0.153346,0.045539,0.337361,0.019517,0.046468
1,3611384,4,Income,Vaccination centre,morning,89266453013ffff,2.217098,0.834146,0.065041,0.091057,0.395122,0.000000,0.000000
2,3611385,4,Income,Vaccination centre,morning,89266453017ffff,0.321941,0.790864,0.146395,0.055036,0.293891,0.014860,0.003853
3,3611386,4,Income,Vaccination centre,morning,8926645301bffff,3.642498,0.433267,0.165339,0.068725,0.215139,0.000000,0.044821
4,3611387,4,Income,Vaccination centre,morning,89266453023ffff,2.879983,0.790864,0.146395,0.055036,0.293891,0.014860,0.003853
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615485,3611378,4,Income,Vaccination centre,morning,89266452eb3ffff,1.863840,0.835317,0.130134,0.079846,0.330902,0.014587,0.014971
4615486,3611379,4,Income,Vaccination centre,morning,89266452eb7ffff,1.863840,0.942454,0.183496,0.018458,0.319218,0.000000,0.000000
4615487,3611380,4,Income,Vaccination centre,morning,89266452ebbffff,0.766955,0.835317,0.130134,0.079846,0.330902,0.014587,0.014971
4615488,3611381,4,Income,Vaccination centre,morning,89266453003ffff,3.642498,0.790864,0.146395,0.055036,0.293891,0.014860,0.003853


In [24]:
df_accessibility.to_csv("accessibilityProcessed.csv")

In [25]:
X = df_accessibility.loc[:, df_accessibility.columns != "accessibility"]
y = df_accessibility['accessibility']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
lr = LinearRegression()
ridge = Ridge(alpha=1.0)
rf = RandomForestRegressor(n_estimators=20, max_depth=10, oob_score=True, ccp_alpha=0.0)
extra = ExtraTreeRegressor(max_depth=10, ccp_alpha=0.0)
model = lr

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("RMSE: " + str(mean_squared_error(y_train, y_pred)))